# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\\$645 - 675) underperformed compared to schools with smaller budgets (\\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

**Note:**
Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import sqlite3

con = sqlite3.connect('/Users/swikar/Desktop/python.db')

school_data = pd.read_sql_query("SELECT * FROM school;", con)
student_data = pd.read_sql_query("SELECT * FROM student;", con)

con.close()

print(school_data.head())
print(student_data.head())


   id  School ID            school_name      type  size   budget
0   0          0      Huang High School  District  2917  1910635
1   1          1   Figueroa High School  District  2949  1884411
2   2          2    Shelton High School   Charter  1761  1056600
3   3          3  Hernandez High School  District  4635  3022020
4   4          4    Griffin High School   Charter  1468   917500
   id  Student ID       student_name gender grade        school_name  \
0   0           0       Paul Bradley      M   9th  Huang High School   
1   1           1       Victor Smith      M  12th  Huang High School   
2   2           2    Kevin Rodriguez      M  12th  Huang High School   
3   3           3  Dr. Richard Scott      M  12th  Huang High School   
4   4           4         Bonnie Ray      F   9th  Huang High School   

   reading_score  math_score  
0             66          79  
1             94          61  
2             90          60  
3             67          58  
4             97      

In [4]:
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,id_x,Student ID,student_name,gender,grade,school_name,reading_score,math_score,id_y,School ID,type,size,budget
0,0,0,Paul Bradley,M,9th,Huang High School,66,79,0,0,District,2917,1910635
1,1,1,Victor Smith,M,12th,Huang High School,94,61,0,0,District,2917,1910635
2,2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,0,District,2917,1910635
3,3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,0,District,2917,1910635
4,4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [5]:
# Create a District Summary




In [7]:
# Total number of schools

total_schools = school_data['school_name'].nunique()



In [ ]:
# Total number of students

total_students = student_data['student_name'].count()



In [ ]:
# Total budget


total_budget = school_data['budget'].sum()


In [21]:
average_math_score = student_data['math_score'].mean()



In [9]:

average_reading_score = student_data['reading_score'].mean()



In [22]:
overall_passing_rate=(average_math_score + average_reading_score) / 2

In [23]:

passing_math_percentage = (
    student_data[student_data['math_score'] >= 70]['student_name'].count() / total_students
) * 100


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [29]:
# Group the student data by school
grouped_students = student_data.groupby('school_name')

total_students_per_school = grouped_students['student_name'].count()

average_math_score_per_school = grouped_students['math_score'].mean()
average_reading_score_per_school = grouped_students['reading_score'].mean()

passing_math = student_data[student_data['math_score'] >= 70].groupby('school_name')['student_name'].count()
passing_reading = student_data[student_data['reading_score'] >= 70].groupby('school_name')['student_name'].count()

passing_math_percentage = (passing_math / total_students_per_school) * 100
passing_reading_percentage = (passing_reading / total_students_per_school) * 100

overall_passing_rate = (passing_math_percentage + passing_reading_percentage) / 2
school_summary = school_data.set_index('school_name').join(pd.DataFrame({
    'Total Students': total_students_per_school,
    'Average Math Score': average_math_score_per_school,
    'Average Reading Score': average_reading_score_per_school,
    '% Passing Math': passing_math_percentage,
    '% Passing Reading': passing_reading_percentage,
    'Overall Passing Rate': overall_passing_rate
}))

school_summary['Per Student Budget'] = school_summary['budget'] / school_summary['Total Students']

school_summary.reset_index(inplace=True)

print(school_summary.head())


             school_name  id  School ID      type  size   budget  \
0      Huang High School   0          0  District  2917  1910635   
1   Figueroa High School   1          1  District  2949  1884411   
2    Shelton High School   2          2   Charter  1761  1056600   
3  Hernandez High School   3          3  District  4635  3022020   
4    Griffin High School   4          4   Charter  1468   917500   

   Total Students  Average Math Score  Average Reading Score  % Passing Math  \
0            2917           76.629414              81.182722       65.683922   
1            2949           76.711767              81.158020       65.988471   
2            1761           83.359455              83.725724       93.867121   
3            4635           77.289752              80.934412       66.752967   
4            1468           83.351499              83.816757       93.392371   

   % Passing Reading  Overall Passing Rate  Per Student Budget  
0          81.316421             73.500171   

### Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [30]:

# Sort and display the top five schools in overall passing rate
top_schools = school_summary.sort_values(by='Overall Passing Rate', ascending=False).head(5)
print("Top Performing Schools (By Overall Passing Rate):")
print(top_schools)



Top Performing Schools (By Overall Passing Rate):
            school_name  id  School ID     type  size   budget  \
6   Cabrera High School   6          6  Charter  1858  1081356   
14   Thomas High School  14         14  Charter  1635  1043130   
9      Pena High School   9          9  Charter   962   585858   
4   Griffin High School   4          4  Charter  1468   917500   
5    Wilson High School   5          5  Charter  2283  1319574   

    Total Students  Average Math Score  Average Reading Score  % Passing Math  \
6             1858           83.061895              83.975780       94.133477   
14            1635           83.418349              83.848930       93.272171   
9              962           83.839917              84.044699       94.594595   
4             1468           83.351499              83.816757       93.392371   
5             2283           83.274201              83.989488       93.867718   

    % Passing Reading  Overall Passing Rate  Per Student Budget  


In [25]:
# Calculate total school budget
total_school_budget = school_data.groupby('school_name')['budget'].sum()
print(total_school_budget)


school_name
Bailey High School       3124928
Cabrera High School      1081356
Figueroa High School     1884411
Ford High School         1763916
Griffin High School       917500
Hernandez High School    3022020
Holden High School        248087
Huang High School        1910635
Johnson High School      3094650
Pena High School          585858
Rodriguez High School    2547363
Shelton High School      1056600
Thomas High School       1043130
Wilson High School       1319574
Wright High School       1049400
Name: budget, dtype: int64


In [31]:
# Calculate per student budget
per_student_budget = total_school_budget / total_students_per_school
print(per_student_budget)




school_name
Bailey High School       628.0
Cabrera High School      582.0
Figueroa High School     639.0
Ford High School         644.0
Griffin High School      625.0
Hernandez High School    652.0
Holden High School       581.0
Huang High School        655.0
Johnson High School      650.0
Pena High School         609.0
Rodriguez High School    637.0
Shelton High School      600.0
Thomas High School       638.0
Wilson High School       578.0
Wright High School       583.0
dtype: float64


In [32]:
# Cacluate the avg math and reading score

average_math_score_per_school = grouped_students['math_score'].mean()
average_reading_score_per_school = grouped_students['reading_score'].mean()

print(average_math_score_per_school)
print(average_reading_score_per_school)



school_name
Bailey High School       77.048432
Cabrera High School      83.061895
Figueroa High School     76.711767
Ford High School         77.102592
Griffin High School      83.351499
Hernandez High School    77.289752
Holden High School       83.803279
Huang High School        76.629414
Johnson High School      77.072464
Pena High School         83.839917
Rodriguez High School    76.842711
Shelton High School      83.359455
Thomas High School       83.418349
Wilson High School       83.274201
Wright High School       83.682222
Name: math_score, dtype: float64
school_name
Bailey High School       81.033963
Cabrera High School      83.975780
Figueroa High School     81.158020
Ford High School         80.746258
Griffin High School      83.816757
Hernandez High School    80.934412
Holden High School       83.814988
Huang High School        81.182722
Johnson High School      80.966394
Pena High School         84.044699
Rodriguez High School    80.744686
Shelton High School      83.72572

#### Find the passing rate for math and reading (above 70 points)

In [34]:
total_math_count = student_data.groupby('school_name')['math_score'].count()
math_passing_count = student_data[student_data['math_score'] >= 70].groupby('school_name')['math_score'].count()
math_passing_rate = (math_passing_count / total_math_count) * 100

print("Math Passing Rate:\n", math_passing_rate)


Math Passing Rate:
 school_name
Bailey High School       66.680064
Cabrera High School      94.133477
Figueroa High School     65.988471
Ford High School         68.309602
Griffin High School      93.392371
Hernandez High School    66.752967
Holden High School       92.505855
Huang High School        65.683922
Johnson High School      66.057551
Pena High School         94.594595
Rodriguez High School    66.366592
Shelton High School      93.867121
Thomas High School       93.272171
Wilson High School       93.867718
Wright High School       93.333333
Name: math_score, dtype: float64


In [35]:
total_read_count = student_data.groupby('school_name')['reading_score'].count()

read_passing_count = student_data[student_data['reading_score'] >= 70].groupby('school_name')['reading_score'].count()

reading_passing_rate = (read_passing_count / total_read_count) * 100

print("Reading Passing Rate:\n", reading_passing_rate)


Reading Passing Rate:
 school_name
Bailey High School       81.933280
Cabrera High School      97.039828
Figueroa High School     80.739234
Ford High School         79.299014
Griffin High School      97.138965
Hernandez High School    80.862999
Holden High School       96.252927
Huang High School        81.316421
Johnson High School      81.222432
Pena High School         95.945946
Rodriguez High School    80.220055
Shelton High School      95.854628
Thomas High School       97.308869
Wilson High School       96.539641
Wright High School       96.611111
Name: reading_score, dtype: float64


In [36]:
overall_passing_rate = (math_passing_rate + reading_passing_rate) / 2

print("Overall Passing Rate:\n", overall_passing_rate)




Overall Passing Rate:
 school_name
Bailey High School       74.306672
Cabrera High School      95.586652
Figueroa High School     73.363852
Ford High School         73.804308
Griffin High School      95.265668
Hernandez High School    73.807983
Holden High School       94.379391
Huang High School        73.500171
Johnson High School      73.639992
Pena High School         95.270270
Rodriguez High School    73.293323
Shelton High School      94.860875
Thomas High School       95.290520
Wilson High School       95.203679
Wright High School       94.972222
dtype: float64


### Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [37]:
# Sort and display the worst five schools in overall passing rate
worst_schools = school_summary.sort_values(by='Overall Passing Rate', ascending=True).head(5)

print("Worst Performing Schools (By Overall Passing Rate):")
print(worst_schools)




Worst Performing Schools (By Overall Passing Rate):
              school_name  id  School ID      type  size   budget  \
11  Rodriguez High School  11         11  District  3999  2547363   
1    Figueroa High School   1          1  District  2949  1884411   
0       Huang High School   0          0  District  2917  1910635   
12    Johnson High School  12         12  District  4761  3094650   
13       Ford High School  13         13  District  2739  1763916   

    Total Students  Average Math Score  Average Reading Score  % Passing Math  \
11            3999           76.842711              80.744686       66.366592   
1             2949           76.711767              81.158020       65.988471   
0             2917           76.629414              81.182722       65.683922   
12            4761           77.072464              80.966394       66.057551   
13            2739           77.102592              80.746258       68.309602   

    % Passing Reading  Overall Passing Rate  P

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [38]:

average_math_by_grade = student_data.groupby(['school_name', 'grade'])['math_score'].mean()

average_math_by_grade_table = average_math_by_grade.reset_index()
print("Average Math Score by School and Grade:")
print(average_math_by_grade_table)




Average Math Score by School and Grade:
              school_name grade  math_score
0      Bailey High School  10th   76.996772
1      Bailey High School  11th   77.515588
2      Bailey High School  12th   76.492218
3      Bailey High School   9th   77.083676
4     Cabrera High School  10th   83.154506
5     Cabrera High School  11th   82.765560
6     Cabrera High School  12th   83.277487
7     Cabrera High School   9th   83.094697
8    Figueroa High School  10th   76.539974
9    Figueroa High School  11th   76.884344
10   Figueroa High School  12th   77.151369
11   Figueroa High School   9th   76.403037
12       Ford High School  10th   77.672316
13       Ford High School  11th   76.918058
14       Ford High School  12th   76.179963
15       Ford High School   9th   77.361345
16    Griffin High School  10th   84.229064
17    Griffin High School  11th   83.842105
18    Griffin High School  12th   83.356164
19    Griffin High School   9th   82.044010
20  Hernandez High School  10th   77

In [41]:
ninth_grade_data = student_data[student_data['grade'] == '9th']

average_math_9th_grade = ninth_grade_data.groupby('school_name')['math_score'].mean()

print("Average Math Score for 9th Grade in Each School:")
print(average_math_9th_grade)




Average Math Score for 9th Grade in Each School:
school_name
Bailey High School       77.083676
Cabrera High School      83.094697
Figueroa High School     76.403037
Ford High School         77.361345
Griffin High School      82.044010
Hernandez High School    77.438495
Holden High School       83.787402
Huang High School        77.027251
Johnson High School      77.187857
Pena High School         83.625455
Rodriguez High School    76.859966
Shelton High School      83.420755
Thomas High School       83.590022
Wilson High School       83.085578
Wright High School       83.264706
Name: math_score, dtype: float64


In [40]:
tenth_grade_data = student_data[student_data['grade'] == '10th']

average_math_10th_grade = tenth_grade_data.groupby('school_name')['math_score'].mean()

print("Average Math Score for 10th Grade in Each School:")
print(average_math_10th_grade)




Average Math Score for 10th Grade in Each School:
school_name
Bailey High School       76.996772
Cabrera High School      83.154506
Figueroa High School     76.539974
Ford High School         77.672316
Griffin High School      84.229064
Hernandez High School    77.337408
Holden High School       83.429825
Huang High School        75.908735
Johnson High School      76.691117
Pena High School         83.372000
Rodriguez High School    76.612500
Shelton High School      82.917411
Thomas High School       83.087886
Wilson High School       83.724422
Wright High School       84.010288
Name: math_score, dtype: float64


In [39]:
eleventh_grade_data = student_data[student_data['grade'] == '11th']

average_math_11th_grade = eleventh_grade_data.groupby('school_name')['math_score'].mean()

print("Average Math Score for 11th Grade in Each School:")
print(average_math_11th_grade)


Average Math Score for 11th Grade in Each School:
school_name
Bailey High School       77.515588
Cabrera High School      82.765560
Figueroa High School     76.884344
Ford High School         76.918058
Griffin High School      83.842105
Hernandez High School    77.136029
Holden High School       85.000000
Huang High School        76.446602
Johnson High School      77.491653
Pena High School         84.328125
Rodriguez High School    76.395626
Shelton High School      83.383495
Thomas High School       83.498795
Wilson High School       83.195326
Wright High School       83.836782
Name: math_score, dtype: float64


In [42]:
twelfth_grade_data = student_data[student_data['grade'] == '12th']

average_math_12th_grade = twelfth_grade_data.groupby('school_name')['math_score'].mean()

print("Average Math Score for 12th Grade in Each School:")
print(average_math_12th_grade)




Average Math Score for 12th Grade in Each School:
school_name
Bailey High School       76.492218
Cabrera High School      83.277487
Figueroa High School     77.151369
Ford High School         76.179963
Griffin High School      83.356164
Hernandez High School    77.186567
Holden High School       82.855422
Huang High School        77.225641
Johnson High School      76.863248
Pena High School         84.121547
Rodriguez High School    77.690748
Shelton High School      83.778976
Thomas High School       83.497041
Wilson High School       83.035794
Wright High School       83.644986
Name: math_score, dtype: float64


### Reading Score by Grade 

* Perform the same operations as above for reading scores

In [ ]:
average_reading_by_grade = student_data.groupby(['school_name', 'grade'])['reading_score'].mean()

average_reading_by_grade_table = average_reading_by_grade.reset_index()

print("Average Reading Score by School and Grade:")
print(average_reading_by_grade_table)



In [ ]:
ninth_grade_reading = student_data[student_data['grade'] == '9th']

average_reading_9th_grade = ninth_grade_reading.groupby('school_name')['reading_score'].mean()

print("Average Reading Score for 9th Grade in Each School:")
print(average_reading_9th_grade)




In [43]:
tenth_grade_reading = student_data[student_data['grade'] == '10th']

average_reading_10th_grade = tenth_grade_reading.groupby('school_name')['reading_score'].mean()

print("Average Reading Score for 10th Grade in Each School:")
print(average_reading_10th_grade)




Average Reading Score for 10th Grade in Each School:
school_name
Bailey High School       80.907183
Cabrera High School      84.253219
Figueroa High School     81.408912
Ford High School         81.262712
Griffin High School      83.706897
Hernandez High School    80.660147
Holden High School       83.324561
Huang High School        81.512386
Johnson High School      80.773431
Pena High School         83.612000
Rodriguez High School    80.629808
Shelton High School      83.441964
Thomas High School       84.254157
Wilson High School       84.021452
Wright High School       83.812757
Name: reading_score, dtype: float64


In [ ]:
eleventh_grade_reading = student_data[student_data['grade'] == '11th']

average_reading_11th_grade = eleventh_grade_reading.groupby('school_name')['reading_score'].mean()

print("Average Reading Score for 11th Grade in Each School:")
print(average_reading_11th_grade)




In [45]:
twelfth_grade_reading = student_data[student_data['grade'] == '12th']
average_reading_12th_grade = twelfth_grade_reading.groupby('school_name')['reading_score'].mean()

print("Average Reading Score for 12th Grade in Each School:")
print(average_reading_12th_grade)




Average Reading Score for 12th Grade in Each School:
school_name
Bailey High School       80.912451
Cabrera High School      84.287958
Figueroa High School     81.384863
Ford High School         80.662338
Griffin High School      84.013699
Hernandez High School    80.857143
Holden High School       84.698795
Huang High School        80.305983
Johnson High School      81.227564
Pena High School         84.591160
Rodriguez High School    80.376426
Shelton High School      82.781671
Thomas High School       83.831361
Wilson High School       84.317673
Wright High School       84.073171
Name: reading_score, dtype: float64


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [48]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [46]:

school_summary['Per Student Budget'] = school_summary['budget'] / school_summary['Total Students']


In [49]:

school_summary['Spending Ranges (Per Student)'] = pd.cut(
    school_summary['Per Student Budget'], spending_bins, labels=group_names
)


In [50]:

avg_math_score_by_spending = school_summary.groupby('Spending Ranges (Per Student)')['Average Math Score'].mean()

print("Average Math Score by Spending Range:")
print(avg_math_score_by_spending)





Average Math Score by Spending Range:
Spending Ranges (Per Student)
<$585       83.455399
$585-615    83.599686
$615-645    79.079225
$645-675    76.997210
Name: Average Math Score, dtype: float64


/var/folders/12/69f7sd750k7cqbxh1trh12th0000gn/T/ipykernel_3402/2409062414.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  avg_math_score_by_spending = school_summary.groupby('Spending Ranges (Per Student)')['Average Math Score'].mean()


In [51]:

math_passing_by_spending = school_summary.groupby('Spending Ranges (Per Student)')['% Passing Math'].mean()

print("Math Passing Percentage by Spending Range:")
print(math_passing_by_spending)




Math Passing Percentage by Spending Range:
Spending Ranges (Per Student)
<$585       93.460096
$585-615    94.230858
$615-645    75.668212
$645-675    66.164813
Name: % Passing Math, dtype: float64


/var/folders/12/69f7sd750k7cqbxh1trh12th0000gn/T/ipykernel_3402/2574247551.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  math_passing_by_spending = school_summary.groupby('Spending Ranges (Per Student)')['% Passing Math'].mean()


In [52]:

reading_passing_by_spending = school_summary.groupby('Spending Ranges (Per Student)')['% Passing Reading'].mean()

print("Reading Passing Percentage by Spending Range:")
print(reading_passing_by_spending)




Reading Passing Percentage by Spending Range:
Spending Ranges (Per Student)
<$585       96.610877
$585-615    95.900287
$615-645    86.106569
$645-675    81.133951
Name: % Passing Reading, dtype: float64


/var/folders/12/69f7sd750k7cqbxh1trh12th0000gn/T/ipykernel_3402/1098314201.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  reading_passing_by_spending = school_summary.groupby('Spending Ranges (Per Student)')['% Passing Reading'].mean()


In [53]:

overall_passing_by_spending = school_summary.groupby('Spending Ranges (Per Student)')['Overall Passing Rate'].mean()

print("Overall Passing Rate by Spending Range:")
print(overall_passing_by_spending)




Overall Passing Rate by Spending Range:
Spending Ranges (Per Student)
<$585       95.035486
$585-615    95.065572
$615-645    80.887391
$645-675    73.649382
Name: Overall Passing Rate, dtype: float64


/var/folders/12/69f7sd750k7cqbxh1trh12th0000gn/T/ipykernel_3402/22288026.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  overall_passing_by_spending = school_summary.groupby('Spending Ranges (Per Student)')['Overall Passing Rate'].mean()


### Scores by School Size

* Perform the same operations as above, based on school size.

In [54]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [60]:
school_summary['School Size'] = pd.cut(school_summary['Total Students'], size_bins, labels=group_names)


Look for the total count of test scores that pass 70% or higher




In [61]:
# math_pass_size


math_pass_size = student_data[student_data['math_score'] >= 70].groupby('school_name')['student_name'].count()

school_summary = school_summary.merge(math_pass_size, on='school_name', how='left').rename(columns={'student_name': 'Math Passing Count'})


In [62]:

read_pass_size = student_data[student_data['reading_score'] >= 70].groupby('school_name')['student_name'].count()

school_summary = school_summary.merge(read_pass_size, on='school_name', how='left').rename(columns={'student_name': 'Reading Passing Count'})



In [70]:
# Check the lengths to ensure alignment
print(len(school_summary))
print(school_summary[['Math Passing Count', 'Reading Passing Count']].head())



15
                       Math Passing Count  Math Passing Count  \
school_name                                                     
Huang High School                    1916                1916   
Figueroa High School                 1946                1946   
Shelton High School                  1653                1653   
Hernandez High School                3094                3094   
Griffin High School                  1371                1371   

                       Reading Passing Count  Reading Passing Count  
school_name                                                          
Huang High School                       2372                   2372  
Figueroa High School                    2381                   2381  
Shelton High School                     1688                   1688  
Hernandez High School                   3748                   3748  
Griffin High School                     1426                   1426  


KeyError: "['school_name'] not in index"

### Scores by School Type

* Perform the same operations as above, based on school type.

In [55]:
# Create bins and groups, school type {'Charter', 'District'}


# Create bins and groups by school type {'Charter', 'District'}
school_type_groups = school_data.groupby('type')


Find counts of the passing 70 or higher score for the both test


In [56]:
# math pass size
# Find the counts of students with math scores 70 or higher, grouped by school type
math_pass_size = student_data[student_data['math_score'] >= 70].groupby('school_name')['student_name'].count()

# Merge the math passing counts with the school summary by school name
school_summary = school_summary.merge(math_pass_size, on='school_name', how='left').rename(columns={'student_name': 'Math Passing Count'})




In [57]:
# reading pass size
# Find the counts of students with reading scores 70 or higher, grouped by school type
reading_pass_size = student_data[student_data['reading_score'] >= 70].groupby('school_name')['student_name'].count()

# Merge the reading passing counts with the school summary by school name
school_summary = school_summary.merge(reading_pass_size, on='school_name', how='left').rename(columns={'student_name': 'Reading Passing Count'})



In [58]:
# Calculate the overall passing rate

# Calculate the overall passing rate by averaging the math and reading passing rates
school_summary['Overall Passing Rate'] = (school_summary['Math Passing Count'] + school_summary['Reading Passing Count']) / 2

# Group the summary data by school type and calculate the average overall passing rate
overall_passing_rate_by_type = school_summary.groupby('type')['Overall Passing Rate'].mean()

# Display the overall passing rate by school type
print("Overall Passing Rate by School Type:")
print(overall_passing_rate_by_type)


Overall Passing Rate by School Type:
type
Charter     1450.687500
District    2840.642857
Name: Overall Passing Rate, dtype: float64
